### OpenAI Functions w/ LangChain and Pydantic

OpenAI's `gpt-3.5-turbo-0613` and `gpt-4-0613` models have been fine-tuned to interact with functions. These advancements enable the models to determine when and how a user-defined function should be invoked based on the context provided in the prompt. It is important to understand that the [Chat Completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api) does not actually execute the `function` itself. Instead, it generates a structured `JSON` response that developers can use to call the `function` within their codebase.

To effectively use `function-calling`, it is required to first define the functions using a `JSON` schema, and then incorporate the `functions` and `function_call` properties in a [Chat Completions](https://platform.openai.com/docs/guides/gpt/chat-completions-api) request.

This notebook illustrates how to combine [LangChain](https://www.langchain.com/) and [Pydantic](https://docs.pydantic.dev/) as an abstraction layer to facilitate the process of creating `OpenAI` `functions` and handling `JSON` formatting.

For comprehensive information, refer to the [OpenAI Function Calling](https://platform.openai.com/docs/guides/gpt/function-calling) and [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/) guides.

#### 1. Load OpenAI API Key

In [1]:
%pip install --upgrade openai langchain python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']


#### 2. Creating OpenAI Functions w/ LangChain and Pydantic

An `OpenAI` `function` is defined by three main parameters: `name`, `description`, and `parameters`. The `description` serves a pivotal role, guiding the model in deciding when and how to execute the function, making it crucial to provide a clear and concise explanation of the function's purpose.

The `parameters` represent a `JSON` schema detailing the expected inputs that the function can accept.

In this section, we will explore how to leverage `Pydantic` and `LangChain` to easily construct `OpenAI` function definitions that include these parameters.

In [3]:
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

##### 2.1 Define Pydantic Models

In this example, we aim to enhance `OpenAI` with the functionality of a travel assistant. To achieve this, we need to invoke two key functions: `FlightSearch` and `EventSearch`. Our first step involves crafting the corresponding `Pydantic` models to define the structure of the data we need to extract from the prompt. In practice. the extracted data will be the arguments of those functions.

[Pydantic Models](https://docs.pydantic.dev/latest/concepts/models/) are essentially classes that derive from `pydantic.BaseModel`, defining fields as type-annotated attributes. They bear a strong resemblance to `Python` dataclasses. However, they have been designed with subtle but significant differences that optimize various operations such as validation, serialization, and `JSON` schema generation. We will delve into these capabilities in the forthcoming step.

In [4]:
class FlightSearch(BaseModel):
    """To retrieve flight results, call this function with: the origin and destination airport codes, and the date of travel."""
    
    origin: str = Field(..., description="origin airport code", examples=["SFO"])
    destination: str = Field(..., description="destination airport code", examples=["LAX"])
    date: str = Field(..., description="date to search flights", examples=["2024-08-01"])

    async def run(self):
        print(f"Searching for flights from {self.origin} to {self.destination} on {self.date}")
        pass

In [5]:
class EventSearch(BaseModel):
    """Retrieve the top events in a city for a given date."""
    
    location: str = Field(..., description="city name", examples=["San Francisco"])
    date: str = Field(..., description="date to search events", examples=["2024-08-01"])
    n: int = Field(..., description="number of events to return", examples=[5])

    async def run(self):
        print(f"Searching for {self.n} events in {self.location} on {self.date}")
        pass

##### 2.2 Convert Pydantic Functions to OpenAI

In [6]:
travel_functions = [
    convert_pydantic_to_openai_function(FlightSearch),
    convert_pydantic_to_openai_function(EventSearch),
]

Now, let's take a look at how our `Pydantic` models have been translated into `OpenAI` function definitions. It's worth noting that the `docstring` for each class is repurposed as the function's description. This underscores the significance of a well-written `docstring`. Additionally, the fields of the model serve as the foundation for defining the function's parameters.

In [7]:
from pprint import pp

for function in travel_functions:
    pp(function)
    print()

{'name': 'FlightSearch',
 'description': 'To retrieve flight results, call this function with: the '
                'origin and destination airport codes, and the date of travel.',
 'parameters': {'description': 'To retrieve flight results, call this function '
                               'with: the origin and destination airport '
                               'codes, and the date of travel.',
                'properties': {'origin': {'description': 'origin airport code',
                                          'examples': ['SFO'],
                                          'title': 'Origin',
                                          'type': 'string'},
                               'destination': {'description': 'destination '
                                                              'airport code',
                                               'examples': ['LAX'],
                                               'title': 'Destination',
                                      

#### 3. Invoking OpenAI Functions w/ Langchain Expression Language

This section illustrates how to compose the input and output types for our functions using the `langchain expression language (LCEL)`.

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

oai = ChatOpenAI(temperature=0)

##### 3.1 Create a Chat Prompt Template

A chat template serves as the foundation for context-aware prompts.

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Your task is to help users find flights and events."),
    ("user", "{query}")
])

##### 3.2 Bind Functions with the Model

 This allows `OpenAI` to intelligently select the appropriate function in response to the user's query.

In [10]:
openai_with_travel_functions = oai.bind(functions=travel_functions)

##### 3.3 Composing Input and Output Functions

Composition in `langchain expression language (LCEL)` follows the syntax of Linux pipes.

In [11]:
travel_assist_chain = prompt | openai_with_travel_functions 

##### 3.4 Invoking Functions

We have reached the stage where we can let `OpenAI` determine which of our functions should be invoked based on the context provided in the prompt.

Note the following example where `OpenAI` follows the instructions provided in the `FlightSearch` description to autonomously parse the date from the user's query, and convert city names into airport codes.

In [13]:
travel_assist_chain.invoke({"query": "search one-way flights from San Francisco to Los Angeles for 22nd January 2024"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "origin": "SFO",\n  "destination": "LAX",\n  "date": "2024-01-22"\n}', 'name': 'FlightSearch'}})

Similarly, `OpenAI` identifies the city name, date, and event count as defined in the `EventSearch` function description.

In [14]:
travel_assist_chain.invoke({"query": "what are the top three events scheduled in Los Angeles for 25th January 2024?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "location": "Los Angeles",\n  "date": "2024-01-25",\n  "n": 3\n}', 'name': 'EventSearch'}})

##### 3.5 Parsing Output as `JSON`

You might want to enhance this worflow to easily parse outputs into `JSON` format by composing also the `JsonOutputFunctionsParser()`.

In [15]:
travel_assist_chain = prompt | openai_with_travel_functions | JsonOutputFunctionsParser()

This facilitates the retrieval of structured `JSON` responses in our examples.

In [16]:
travel_assist_chain.invoke({"query": "search one-way flights from San Francisco to Los Angeles for 22nd January 2024"})

{'origin': 'SFO', 'destination': 'LAX', 'date': '2024-01-22'}

In [17]:
travel_assist_chain.invoke({"query": "what are the top three events scheduled in Los Angeles for 25th January 2024?"})

{'location': 'Los Angeles', 'date': '2024-01-25', 'n': 3}

----